## 練習時間
參考 Day 12 範例程式，離散化你覺得有興趣的欄位，並嘗試找出有趣的訊息

In [1]:
# Import 需要的套件
import os
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

### 之前做過的處理

In [2]:
# 設定 data_path
dir_data = './data/'
f_app_train = os.path.join(dir_data, 'application_train.csv')
f_app_test = os.path.join(dir_data, 'application_test.csv')

app_train = pd.read_csv(f_app_train)
app_test = pd.read_csv(f_app_test)

from sklearn.preprocessing import LabelEncoder

# Create a label encoder object
le = LabelEncoder()
le_count = 0

# Iterate through the columns
for col in app_train:
    if app_train[col].dtype == 'object':
        # If 2 or fewer unique categories
        if len(list(app_train[col].unique())) <= 2:
            # Train on the training data
            le.fit(app_train[col])
            # Transform both training and testing data
            app_train[col] = le.transform(app_train[col])
            app_test[col] = le.transform(app_test[col])
            
            # Keep track of how many columns were label encoded
            le_count += 1
            
app_train = pd.get_dummies(app_train)
app_test = pd.get_dummies(app_test)

# Create an anomalous flag column
app_train['DAYS_EMPLOYED_ANOM'] = app_train["DAYS_EMPLOYED"] == 365243
app_train['DAYS_EMPLOYED'].replace({365243: np.nan}, inplace = True)
# also apply to testing dataset
app_test['DAYS_EMPLOYED_ANOM'] = app_test["DAYS_EMPLOYED"] == 365243
app_test["DAYS_EMPLOYED"].replace({365243: np.nan}, inplace = True)

# absolute the value of DAYS_BIRTH
app_train['DAYS_BIRTH'] = abs(app_train['DAYS_BIRTH'])
app_test['DAYS_BIRTH'] = abs(app_test['DAYS_BIRTH'])

In [3]:
## 列出欄位值項目>100 的連續型變數

In [4]:
for i in app_train.columns:
    if len(app_train[i].unique()) > 100:
        print(f"{i} : {len(app_train[i].unique())}")

SK_ID_CURR : 307511
AMT_INCOME_TOTAL : 2548
AMT_CREDIT : 5603
AMT_ANNUITY : 13673
AMT_GOODS_PRICE : 1003
DAYS_BIRTH : 17460
DAYS_EMPLOYED : 12574
DAYS_REGISTRATION : 15688
DAYS_ID_PUBLISH : 6168
EXT_SOURCE_1 : 114585
EXT_SOURCE_2 : 119832
EXT_SOURCE_3 : 815
APARTMENTS_AVG : 2340
BASEMENTAREA_AVG : 3781
YEARS_BEGINEXPLUATATION_AVG : 286
YEARS_BUILD_AVG : 150
COMMONAREA_AVG : 3182
ELEVATORS_AVG : 258
ENTRANCES_AVG : 286
FLOORSMAX_AVG : 404
FLOORSMIN_AVG : 306
LANDAREA_AVG : 3528
LIVINGAPARTMENTS_AVG : 1869
LIVINGAREA_AVG : 5200
NONLIVINGAPARTMENTS_AVG : 387
NONLIVINGAREA_AVG : 3291
APARTMENTS_MODE : 761
BASEMENTAREA_MODE : 3842
YEARS_BEGINEXPLUATATION_MODE : 222
YEARS_BUILD_MODE : 155
COMMONAREA_MODE : 3129
LANDAREA_MODE : 3564
LIVINGAPARTMENTS_MODE : 737
LIVINGAREA_MODE : 5302
NONLIVINGAPARTMENTS_MODE : 168
NONLIVINGAREA_MODE : 3328
APARTMENTS_MEDI : 1149
BASEMENTAREA_MEDI : 3773
YEARS_BEGINEXPLUATATION_MEDI : 246
YEARS_BUILD_MEDI : 152
COMMONAREA_MEDI : 3203
LANDAREA_MEDI : 3561
LIVING

- DAYS_BIRTH 客戶年齡
- DAYS_EMPLOYED 客戶工作年資

In [5]:
app_train[["DAYS_BIRTH","DAYS_EMPLOYED"]].describe().T

,count,mean,std,min,25%,50%,75%,max
DAYS_BIRTH,307511.0,16036.995067,4363.988632,7489.0,12413.0,15750.0,19682.0,25229.0
DAYS_EMPLOYED,252137.0,-2384.169325,2338.360162,-17912.0,-3175.0,-1648.0,-767.0,0.0


In [6]:
app_train["DAYS_BIRTH_year"]=app_train["DAYS_BIRTH"]/365

In [7]:
app_train["DAYS_EMPLOYED_year"] = app_train["DAYS_EMPLOYED"]/(-365)

In [8]:
app_train[["DAYS_BIRTH_year","DAYS_EMPLOYED_year"]].describe().T

,count,mean,std,min,25%,50%,75%,max
DAYS_BIRTH_year,307511.0,43.936973,11.956133,20.517808,34.008219,43.150685,53.923288,69.120548
DAYS_EMPLOYED_year,252137.0,6.531971,6.406466,-0.000000,2.101370,4.515068,8.698630,49.073973


### 等寬劃分(ew)

In [16]:
app_train["DAYS_BIRTH_year_ew"]=pd.cut(app_train["DAYS_BIRTH_year"],10)
app_train["DAYS_BIRTH_year_ew"].value_counts().sort_index()

(20.469, 25.378]    13679
(25.378, 30.238]    33127
(30.238, 35.099]    38430
(35.099, 39.959]    41758
(39.959, 44.819]    40350
(44.819, 49.679]    34311
(49.679, 54.54]     33544
(54.54, 59.4]       32650
(59.4, 64.26]       27685
(64.26, 69.121]     11977
Name: DAYS_BIRTH_year_ew, dtype: int64

### 等頻劃分(ef)

In [15]:
app_train["DAYS_EMPLOYED_year_ef"]=pd.qcut(app_train["DAYS_EMPLOYED_year"],4)
app_train["DAYS_EMPLOYED_year_ef"].value_counts().sort_index()

(-0.001, 2.101]    63056
(2.101, 4.515]     63068
(4.515, 8.699]     62980
(8.699, 49.074]    63033
Name: DAYS_EMPLOYED_year_ef, dtype: int64

## 自定義的 bin

In [20]:
app_train["DAYS_BIRTH_year_sb"]=pd.cut(app_train["DAYS_BIRTH_year"],[20,30,40,50,60,70])
app_train["DAYS_BIRTH_year_sb"].value_counts().sort_index()

(20, 30]    45021
(30, 40]    82308
(40, 50]    76541
(50, 60]    68062
(60, 70]    35579
Name: DAYS_BIRTH_year_sb, dtype: int64

In [23]:
app_train["DAYS_EMPLOYED_year_sb"]=pd.cut(app_train["DAYS_EMPLOYED_year"],[0,1,3,5,10,20,50])
app_train["DAYS_EMPLOYED_year_sb"].value_counts().sort_index()

(0, 1]      27980
(1, 3]      61472
(3, 5]      46857
(5, 10]     64872
(10, 20]    38398
(20, 50]    12556
Name: DAYS_EMPLOYED_year_sb, dtype: int64